In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
import scikit_posthocs as sp



In [2]:
path = '/export/usuarios_ml4ds/ammesa/Data/4_indexed_data/res/LDA_toy_dir'
df_weighted = pd.read_csv(os.path.join(path, 'modifica.csv'), sep=';')
df_re = pd.read_parquet(os.path.join(path, 'full_metrics.parquet'))

In [3]:
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI
0,0,toy_questions,TB_ENN,mrr@3,"0,3","0,247922569"
1,1,toy_questions,TB_ENN,mrr@5,"0,325","0,240549579"
2,2,toy_questions,TB_ENN,mrr@10,"0,261402116","0,146204454"
3,3,toy_questions,TB_ENN,precision@3,"0,133333333","0,101213965"
4,4,toy_questions,TB_ENN,precision@5,"0,1","0,061980642"
...,...,...,...,...,...,...
223,223,toy_questions,ANN,hit@10,"0,2","0,247922569"
224,224,toy_questions,ANN,rank_hit@3,"0,1","0,185941926"
225,225,toy_questions,ANN,rank_hit@5,"0,1","0,185941926"
226,226,toy_questions,ANN,rank_hit@10,"0,116666667","0,185079082"


In [4]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,hit_3,rank_hit_3,mrr_5,precision_5,recall_5,ndcg_5,...,mrr_10,precision_10,recall_10,ndcg_10,hit_10,rank_hit_10,time,retrieval_method,thr,weighted
0,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,1.457681,TB_ENN,0,True
1,1.0,0.333333,0.50,1.00000,1,1.0,1.000,0.2,0.50,1.000000,...,0.571429,0.2,1.00,0.817530,1,1.000000,1.464653,TB_ENN,0,True
2,0.5,0.333333,0.25,0.63093,1,0.5,0.500,0.2,0.25,0.630930,...,0.500000,0.1,0.25,0.630930,1,0.500000,1.455168,TB_ENN,0,True
3,1.0,0.333333,0.20,1.00000,0,0.0,1.000,0.2,0.20,1.000000,...,0.583333,0.2,0.40,0.831555,0,0.000000,1.455240,TB_ENN,0,True
4,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,1.458415,TB_ENN,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,0.000000,0.00,0.00000,0,0.0,0.225,0.4,0.40,0.501266,...,0.225000,0.2,0.40,0.501266,0,0.000000,0.008975,ANN,1,True
116,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.009375,ANN,1,True
117,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.166667,0.1,1.00,0.356207,1,0.166667,0.009002,ANN,1,True
118,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.009018,ANN,1,True


In [5]:
def get_significance_marker( p_value):
    if p_value < 0.001:
        return "***"
    elif p_value < 0.01:
        return "**"
    elif p_value < 0.05:
        return "*"
    else:
        return ""
    
def format_metric(metric):
    return (
            "MRR@3" if metric == "mrr_3" else
            "MRR@5" if metric == "mrr_5" else
            "NDCG@3" if metric == "ndcg_3" else
            "NDCG@5" if metric == "ndcg_5" else
            "Precision@3" if metric == "precision_3" else
            "Precision@5" if metric == "precision_5" else
            "Recall@3" if metric == "recall_3" else
            "Recall@5" if metric == "recall_5" else
            "RHit@3" if metric == "rank_hit_3" else
            "RHit@5" if metric == "rank_hit_5" else
            "Time (s)" if "time" in metric else
            metric  # Fallback si no está en la lista
        )
        

def check_normality( values):
    stat, p_value = stats.shapiro(values)
    return p_value > 0.05  

def perform_anova( groups):
    f_stat, p_value = stats.f_oneway(*groups)
    return f_stat, p_value

def perform_kruskal_wallis( groups):
    h_stat, p_value = stats.kruskal(*groups)
    return h_stat, p_value

# Normality tests:

In [6]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,hit_3,rank_hit_3,mrr_5,precision_5,recall_5,ndcg_5,...,mrr_10,precision_10,recall_10,ndcg_10,hit_10,rank_hit_10,time,retrieval_method,thr,weighted
0,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,1.457681,TB_ENN,0,True
1,1.0,0.333333,0.50,1.00000,1,1.0,1.000,0.2,0.50,1.000000,...,0.571429,0.2,1.00,0.817530,1,1.000000,1.464653,TB_ENN,0,True
2,0.5,0.333333,0.25,0.63093,1,0.5,0.500,0.2,0.25,0.630930,...,0.500000,0.1,0.25,0.630930,1,0.500000,1.455168,TB_ENN,0,True
3,1.0,0.333333,0.20,1.00000,0,0.0,1.000,0.2,0.20,1.000000,...,0.583333,0.2,0.40,0.831555,0,0.000000,1.455240,TB_ENN,0,True
4,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,1.458415,TB_ENN,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,0.000000,0.00,0.00000,0,0.0,0.225,0.4,0.40,0.501266,...,0.225000,0.2,0.40,0.501266,0,0.000000,0.008975,ANN,1,True
116,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.009375,ANN,1,True
117,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.166667,0.1,1.00,0.356207,1,0.166667,0.009002,ANN,1,True
118,0.0,0.000000,0.00,0.00000,0,0.0,0.000,0.0,0.00,0.000000,...,0.000000,0.0,0.00,0.000000,0,0.000000,0.009018,ANN,1,True


In [7]:
    
df_weighted["BestValue"] = df_weighted.groupby(["FileID", "Metric"])["Weighted Mean"].transform(
    lambda x: x.min() if "Time (s)" in x.name else x.max()
)
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,"0,3","0,247922569","0,358333333"
1,1,toy_questions,TB_ENN,mrr@5,"0,325","0,240549579","0,335833333"
2,2,toy_questions,TB_ENN,mrr@10,"0,261402116","0,146204454","0,29207672"
3,3,toy_questions,TB_ENN,precision@3,"0,133333333","0,101213965","0,2"
4,4,toy_questions,TB_ENN,precision@5,"0,1","0,061980642","0,16"
...,...,...,...,...,...,...,...
223,223,toy_questions,ANN,hit@10,"0,2","0,247922569","0,5"
224,224,toy_questions,ANN,rank_hit@3,"0,1","0,185941926","0,3"
225,225,toy_questions,ANN,rank_hit@5,"0,1","0,185941926","0,3"
226,226,toy_questions,ANN,rank_hit@10,"0,116666667","0,185079082","0,327777778"


In [8]:
df_weighted = df_weighted[~df_weighted['Metric'].isin(['hit@3', 'hit@5','hit@10','rank_hit@5', 'rank_hit@3','rank_hit@10' ])]
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,"0,3","0,247922569","0,358333333"
1,1,toy_questions,TB_ENN,mrr@5,"0,325","0,240549579","0,335833333"
2,2,toy_questions,TB_ENN,mrr@10,"0,261402116","0,146204454","0,29207672"
3,3,toy_questions,TB_ENN,precision@3,"0,133333333","0,101213965","0,2"
4,4,toy_questions,TB_ENN,precision@5,"0,1","0,061980642","0,16"
...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,"0,165","0,191062304","0,52"
218,218,toy_questions,ANN,ndcg@3,"0,1","0,185941926","0,413092975"
219,219,toy_questions,ANN,ndcg@5,"0,150126584","0,198520768","0,41484739"
220,220,toy_questions,ANN,ndcg@10,"0,185747302","0,199220323","0,474496092"


In [9]:
df_re['Method'] = df_re['retrieval_method'].astype(str)

# Add '_W' if weighted is True
df_re['Method'] += df_re['weighted'].apply(lambda x: '-W' if x else '')

# Add '_V' if thr == 1
df_re['Method'] += df_re['thr'].apply(lambda x: '-V' if x == 1 else '')


In [10]:
df_re = df_re.drop(columns=['hit_3', 'hit_5', 'hit_10',
                            'rank_hit_3', 'rank_hit_5', 'rank_hit_10',])

In [11]:
df_re

,mrr_3,precision_3,recall_3,ndcg_3,mrr_5,precision_5,recall_5,ndcg_5,mrr_10,precision_10,recall_10,ndcg_10,time,retrieval_method,thr,weighted,Method
0,0.0,0.000000,0.00,0.00000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,1.457681,TB_ENN,0,True,TB_ENN-W
1,1.0,0.333333,0.50,1.00000,1.000,0.2,0.50,1.000000,0.571429,0.2,1.00,0.817530,1.464653,TB_ENN,0,True,TB_ENN-W
2,0.5,0.333333,0.25,0.63093,0.500,0.2,0.25,0.630930,0.500000,0.1,0.25,0.630930,1.455168,TB_ENN,0,True,TB_ENN-W
3,1.0,0.333333,0.20,1.00000,1.000,0.2,0.20,1.000000,0.583333,0.2,0.40,0.831555,1.455240,TB_ENN,0,True,TB_ENN-W
4,0.0,0.000000,0.00,0.00000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,1.458415,TB_ENN,0,True,TB_ENN-W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,0.000000,0.00,0.00000,0.225,0.4,0.40,0.501266,0.225000,0.2,0.40,0.501266,0.008975,ANN,1,True,ANN-W-V
116,0.0,0.000000,0.00,0.00000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.009375,ANN,1,True,ANN-W-V
117,0.0,0.000000,0.00,0.00000,0.000,0.0,0.00,0.000000,0.166667,0.1,1.00,0.356207,0.009002,ANN,1,True,ANN-W-V
118,0.0,0.000000,0.00,0.00000,0.000,0.0,0.00,0.000000,0.000000,0.0,0.00,0.000000,0.009018,ANN,1,True,ANN-W-V


In [12]:
# Step 2: Drop the old method and modifiers if you don't need them
df_re = df_re.drop(columns=['retrieval_method', 'thr', 'weighted'])

# Step 3: Melt the dataframe so that metric names become a column, and values are aligned accordingly
df_melted = df_re.melt(id_vars=['Method'], var_name='Metric', value_name='Value')

df_melted

,Method,Metric,Value
0,TB_ENN-W,mrr_3,0.000000
1,TB_ENN-W,mrr_3,1.000000
2,TB_ENN-W,mrr_3,0.500000
3,TB_ENN-W,mrr_3,1.000000
4,TB_ENN-W,mrr_3,0.000000
...,...,...,...
1555,ANN-W-V,time,0.008975
1556,ANN-W-V,time,0.009375
1557,ANN-W-V,time,0.009002
1558,ANN-W-V,time,0.009018


In [13]:
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,"0,3","0,247922569","0,358333333"
1,1,toy_questions,TB_ENN,mrr@5,"0,325","0,240549579","0,335833333"
2,2,toy_questions,TB_ENN,mrr@10,"0,261402116","0,146204454","0,29207672"
3,3,toy_questions,TB_ENN,precision@3,"0,133333333","0,101213965","0,2"
4,4,toy_questions,TB_ENN,precision@5,"0,1","0,061980642","0,16"
...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,"0,165","0,191062304","0,52"
218,218,toy_questions,ANN,ndcg@3,"0,1","0,185941926","0,413092975"
219,219,toy_questions,ANN,ndcg@5,"0,150126584","0,198520768","0,41484739"
220,220,toy_questions,ANN,ndcg@10,"0,185747302","0,199220323","0,474496092"


In [14]:
# Replace commas with dots and convert last 3 columns to float
df_weighted.iloc[:, -3:] = df_weighted.iloc[:, -3:].apply(
    lambda col: pd.to_numeric(col.str.replace(',', '.', regex=False), errors='coerce')
)
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue
0,0,toy_questions,TB_ENN,mrr@3,0.3,0.247923,0.358333
1,1,toy_questions,TB_ENN,mrr@5,0.325,0.24055,0.335833
2,2,toy_questions,TB_ENN,mrr@10,0.261402,0.146204,0.292077
3,3,toy_questions,TB_ENN,precision@3,0.133333,0.101214,0.2
4,4,toy_questions,TB_ENN,precision@5,0.1,0.061981,0.16
...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,0.165,0.191062,0.52
218,218,toy_questions,ANN,ndcg@3,0.1,0.185942,0.413093
219,219,toy_questions,ANN,ndcg@5,0.150127,0.198521,0.414847
220,220,toy_questions,ANN,ndcg@10,0.185747,0.19922,0.474496


In [15]:
def format_weighted_value(row):
    mean = f"{row['Weighted Mean']:.3f}"
    ci = f"{row['95% CI']:.3f}"
    value = f"{mean} \pm {ci}"

    metric = row["Metric"]
    method = row["Method"]

    """
    # Agregar '*' si Kruskal-Wallis indica diferencias significativas
    if metric in significance_dict["Kruskal-Wallis p-value"]:
        kw_p_value = significance_dict["Kruskal-Wallis p-value"][metric]
        if kw_p_value < 0.05:
            value += " *"  # Indica que hay diferencias globales
    """
    significance_marker = ""
    if metric in significance_dict["Significant post-hoc tests (Dunn)"]:
        dunn_df = significance_dict["Significant post-hoc tests (Dunn)"][metric]
        for other_method in dunn_df.index:
            if method in dunn_df.columns and other_method in dunn_df.index:
                p_val = dunn_df.loc[other_method, method]
                if p_val < 0.001:
                    significance_marker = "***"
                elif p_val < 0.01:
                    significance_marker = "**"
                elif p_val < 0.05:
                    significance_marker = "*"

        value += f"^{{{significance_marker}}}"
    
    if row["Weighted Mean"] == row["BestValue"]:
        return f"\\(\\boldsymbol{{{value}}}\\)"
    else:
        return f"\\({value}\\)"

df_long = df_melted
df_weighted['Weighted Mean'] = pd.to_numeric(df_weighted['Weighted Mean'], errors='coerce')
df_weighted['95% CI'] = pd.to_numeric(df_weighted['95% CI'], errors='coerce')
normality_tests = {}
print(df_weighted)
for metric in df_long["Metric"].unique():
    for method in df_long["Method"].unique():
        values = df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"]
        values = df_long[
            (df_long["Metric"] == metric) & (df_long["Method"] == method)
        ]["Value"]

        # Clean and convert to float
        values = pd.to_numeric(values, errors="coerce").dropna()
        if len(values) > 3:  # Shapiro necesita al menos unos pocos valores
            stat, p_value = stats.shapiro(values)
            normality_tests[(metric, method)]=  p_value

# ANOVA or Kruskal-Wallis
anova_results = {}
kruskal_results = {}

for metric in df_long["Metric"].unique():
    groups = [df_long[(df_long["Metric"] == metric) & (df_long["Method"] == method)]["Value"].dropna() for method in df_long["Method"].unique()]
    groups = [pd.to_numeric(g, errors="coerce").dropna() for g in groups]
    # if normality in at least one group
    normal = all(p > 0.05 for p in [normality_tests.get((metric, method), 0) for method in df_long["Method"].unique()])
    
    if normal:
        stat, p_value = stats.f_oneway(*groups)
        anova_results[metric] = p_value
    else:
        stat, p_value = stats.kruskal(*groups)
        kruskal_results[metric] = p_value

# post-hoc tests if Kruskal-Wallis is significant
posthoc_results = {}
for metric, p in kruskal_results.items():
    if p < 0.05:  # Si hay diferencias significativas
        df_filtered = df_long[df_long["Metric"] == metric]
        posthoc = sp.posthoc_dunn(df_filtered, val_col="Value", group_col="Method", p_adjust="bonferroni")
        posthoc_results[metric] = posthoc


    significance_dict = {
        "ANOVA p-value": anova_results,
        "Kruskal-Wallis p-value": kruskal_results,
        "Significant post-hoc tests (Dunn)": posthoc_results
    }
    df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
    
    df_weighted_pivot = df_weighted.pivot_table(index=["FileID", "Method"], columns="Metric", values="Weighted Mean ± CI", aggfunc="first")
    
    latex_output = "\\begin{table*}[h]\n\\centering\n\\resizebox{\\textwidth}{!}{%\n\\begin{tabular}{lc" + "c" * len(df_weighted_pivot.columns) + "}\n\\arrayrulecolor{black}\n\\toprule\n"
    
    latex_output += "File & \\textbf{Method} & " + " & ".join([f"\\textbf{{{col}}}" for col in df_weighted_pivot.columns]) + " \\\\\n\\midrule\n"
    
    current_file_id = None 
    for (file_idx, method), row in df_weighted_pivot.iterrows():
        if file_idx != current_file_id:  
            if current_file_id is not None:
                latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"

            col_n = str(len(df_weighted_pivot.columns) + 2)
            file_id = str(file_idx+'1')
            
            latex_output += f"\\multicolumn{{{col_n}}}{{l}}{{\\textbf{{File {file_id}}}}} \\\\\n"
            
            latex_output += "\\arrayrulecolor{black}\\specialrule{0.5pt}{0pt}{0pt}\\arrayrulecolor{black}\n"
            
            current_file_id = file_idx  
        
        row_start = "\\rowcolor{tableblue} " if method == "XX" else ""
        
        latex_output += row_start + f"& \\textbf{{{method}}} & " + " & ".join(row.astype(str)) + " \\\\\n"
    
    latex_output += "\\bottomrule\n\\end{tabular}}\n\\caption{Performance Metrics per Method for Each File (Best values in bold)}\n\\label{tab:grouped_results}\n\\end{table*}"
    
    print(latex_output)


<>:4: SyntaxWarning: invalid escape sequence '\p'
<>:4: SyntaxWarning: invalid escape sequence '\p'
/tmp/ipykernel_932532/146575848.py:4: SyntaxWarning: invalid escape sequence '\p'
  value = f"{mean} \pm {ci}"
/tmp/ipykernel_932532/146575848.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted['Weighted Mean'] = pd.to_numeric(df_weighted['Weighted Mean'], errors='coerce')
/tmp/ipykernel_932532/146575848.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted['95% CI'] = pd.to_numeric(d

     Unnamed: 0         FileID  Method       Metric  Weighted Mean    95% CI  \
0             0  toy_questions  TB_ENN        mrr@3       0.300000  0.247923   
1             1  toy_questions  TB_ENN        mrr@5       0.325000  0.240550   
2             2  toy_questions  TB_ENN       mrr@10       0.261402  0.146204   
3             3  toy_questions  TB_ENN  precision@3       0.133333  0.101214   
4             4  toy_questions  TB_ENN  precision@5       0.100000  0.061981   
..          ...            ...     ...          ...            ...       ...   
217         217  toy_questions     ANN    recall@10       0.165000  0.191062   
218         218  toy_questions     ANN       ndcg@3       0.100000  0.185942   
219         219  toy_questions     ANN       ndcg@5       0.150127  0.198521   
220         220  toy_questions     ANN      ndcg@10       0.185747  0.199220   
227         227  toy_questions     ANN         time       0.009108  0.000130   

    BestValue  
0    0.358333  
1    0.

/tmp/ipykernel_932532/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_932532/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_932532/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

\begin{table*}[h]
\centering
\resizebox{\textwidth}{!}{%
\begin{tabular}{lcccccccccccccc}
\arrayrulecolor{black}
\toprule
File & \textbf{Method} & \textbf{mrr@10} & \textbf{mrr@3} & \textbf{mrr@5} & \textbf{ndcg@10} & \textbf{ndcg@3} & \textbf{ndcg@5} & \textbf{precision@10} & \textbf{precision@3} & \textbf{precision@5} & \textbf{recall@10} & \textbf{recall@3} & \textbf{recall@5} & \textbf{time} \\
\midrule
\multicolumn{15}{l}{\textbf{File toy_questions1}} \\
\arrayrulecolor{black}\specialrule{0.5pt}{0pt}{0pt}\arrayrulecolor{black}
& \textbf{ANN} & \(0.139 \pm 0.184\) & \(0.100 \pm 0.186\) & \(0.122 \pm 0.186\) & \(0.186 \pm 0.199\) & \(0.100 \pm 0.186\) & \(0.150 \pm 0.199\) & \(0.040 \pm 0.041\) & \(0.033 \pm 0.062\) & \(0.060 \pm 0.079\) & \(0.165 \pm 0.191\) & \(0.025 \pm 0.046\) & \(0.065 \pm 0.083\) & \(0.010 \pm 0.003\) \\
& \textbf{ENN} & \(\boldsymbol{0.292 \pm 0.193}\) & \(\boldsymbol{0.358 \pm 0.251}\) & \(\boldsymbol{0.336 \pm 0.242}\) & \(\boldsymbol{0.474 \pm 0.227}\) & \

/tmp/ipykernel_932532/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)
/tmp/ipykernel_932532/146575848.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weighted["Weighted Mean ± CI"] = df_weighted.apply(format_weighted_value, axis=1)


In [16]:
significance_dict['Significant post-hoc tests (Dunn)']

{'mrr_3':             ANN-W  ANN-W-V     ENN-W   ENN-W-V    TB_ANN  TB_ANN-V  TB_ANN-W  \
 ANN-W         1.0      1.0  1.000000  1.000000  1.000000  1.000000  1.000000   
 ANN-W-V       1.0      1.0  1.000000  1.000000  1.000000  1.000000  1.000000   
 ENN-W         1.0      1.0  1.000000  1.000000  0.745535  0.745535  0.745535   
 ENN-W-V       1.0      1.0  1.000000  1.000000  0.745535  0.745535  0.745535   
 TB_ANN        1.0      1.0  0.745535  0.745535  1.000000  1.000000  1.000000   
 TB_ANN-V      1.0      1.0  0.745535  0.745535  1.000000  1.000000  1.000000   
 TB_ANN-W      1.0      1.0  0.745535  0.745535  1.000000  1.000000  1.000000   
 TB_ANN-W-V    1.0      1.0  0.745535  0.745535  1.000000  1.000000  1.000000   
 TB_ENN        1.0      1.0  1.000000  1.000000  0.745535  0.745535  0.745535   
 TB_ENN-V      1.0      1.0  1.000000  1.000000  1.000000  1.000000  1.000000   
 TB_ENN-W      1.0      1.0  1.000000  1.000000  1.000000  1.000000  1.000000   
 TB_ENN-W-V    1.0 

In [17]:
df_weighted

,Unnamed: 0,FileID,Method,Metric,Weighted Mean,95% CI,BestValue,Weighted Mean ± CI
0,0,toy_questions,TB_ENN,mrr@3,0.300000,0.247923,0.358333,\(0.300 \pm 0.248\)
1,1,toy_questions,TB_ENN,mrr@5,0.325000,0.240550,0.335833,\(0.325 \pm 0.241\)
2,2,toy_questions,TB_ENN,mrr@10,0.261402,0.146204,0.292077,\(0.261 \pm 0.146\)
3,3,toy_questions,TB_ENN,precision@3,0.133333,0.101214,0.2,\(0.133 \pm 0.101\)
4,4,toy_questions,TB_ENN,precision@5,0.100000,0.061981,0.16,\(0.100 \pm 0.062\)
...,...,...,...,...,...,...,...,...
217,217,toy_questions,ANN,recall@10,0.165000,0.191062,0.52,\(0.165 \pm 0.191\)
218,218,toy_questions,ANN,ndcg@3,0.100000,0.185942,0.413093,\(0.100 \pm 0.186\)
219,219,toy_questions,ANN,ndcg@5,0.150127,0.198521,0.414847,\(0.150 \pm 0.199\)
220,220,toy_questions,ANN,ndcg@10,0.185747,0.199220,0.474496,\(0.186 \pm 0.199\)
